# Introduction

> Quantum computing is an emergent field of computer science and engineering that harnesses the unique qualities of quantum mechanics to solve problems beyond the ability of even the most powerful classical computers. - **IBM**
> 

A quantum computer is a type of computer that exploits the principles and phenomena of quantum mechanics to perform complex calculations otherwise intractable with classical computers. There are various physical systems (or technologies) currently being explored for use as qubits including photons, trapped ions, superconducting circuits and spins in semiconductors. Qubits, in these technologies, are realized as fragile quantum systems.

One problem that is shared between all these technologies is the difficulty to adequately isolate the qubits from the external noise: a phenomenon that disrupts the qubits resulting in inevitable errors during quantum computation. Bits in classical computers, in this regard, are different as they are typically realized as the on/off states of transistor switches and represented by their voltage levels with billions of electrons involved. This robustness guarantees near-complete elimination of failures at the physical level.

For quantum computers, there is no such inherent protection, as such, any quantum computer circuit model requires active error correction.

This notebook, through the use of the Qiskit framework, aims to present the theory and the application of the quantum error correction (QEC): a set of techniques used in quantum computing to protect quantum information.

We first begin by installing and importing the necessary tools.

In [ ]:
%pip install qiskit[visualization]
%pip install qiskit-aer

In [8]:
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit, transpile
from qiskit_aer import Aer
from qiskit.quantum_info import random_statevector, Statevector, state_fidelity
from qiskit.visualization import plot_bloch_multivector
from random import randint

# Foundations of quantum error correction

## Quantum errors digitization

The general qubit state is defined as follows

$$
\begin{equation}
|\psi\rangle = \alpha|0\rangle + \beta|1\rangle
\end{equation}
\tag{1}
$$

where $\alpha$ and $\beta$ are complex numbers that satisfy the condition $|\alpha|^2 + |\beta|^2 = 1$.

The previous equation can be rewritten in terms of the geometric representation of the qubit as follows

$$
\begin{equation}
|\psi\rangle = \cos\frac{\theta}{2}|0\rangle + e^{i\phi}\sin\frac{\theta}{2}|1\rangle
\end{equation}
\tag{2}
$$

In this form, the qubit state corresponds to a point and its position is specified by the angles $\theta$ and $\phi$ on the surface of the Bloch sphere. Note that, as in the equation $(1)$, the condition
$|\cos\frac{\theta}{2}|^2+ |e^{i\phi}\sin\frac{\theta}{2}|^2 = 1$ still holds.

The general qubit state can assume a continuum of values between its basis states, from this follows that the qubit is subject to an infinite number of errors.

Qubits errors can occur by a variety of physical processes. In this notebook we analyze the errors which cause the qubit to coherently rotate from one point on the Bloch sphere to another.

Mathematically, coherent errors can be described by a unitary operator $U(\delta\theta, \delta\phi)$ which transforms the qubit states as follows

$$
\begin{equation}
U(\delta\theta, \delta\phi)|\phi\rangle = \cos\frac{\theta + \delta\theta}{2}|0\rangle + e^{i(\phi + \delta\phi)}\sin\frac{\theta + \delta\theta}{2}|1\rangle
\end{equation}
\tag{3}
$$

where $\theta + \delta\theta$ and $\phi + \delta\phi$ are the new coordinates on the Bloch sphere.

From this, we can see that qubits are prone to a continuum of coherent errors obtained by varying the parameters $\delta\theta$ and $\delta\phi$.

It seems we might need a technique to account for a near-infinite amount of error, however, quantum errors can be digitized so that the ability to correct for a finite set of errors is sufficient to correct for any error. To see how this is possible, we first note that coherent noise processes are described by matrices that can be expanded terms of a Pauli basis.

Thus, the single qubit-coherent error process described in the equation $(3)$ can be described in the Pauli basis for two-dimensional matrices as follows

$$
\begin{equation}
U(\delta\theta, \delta\phi)|\phi\rangle =a_II|\psi\rangle + a_XX|\psi\rangle +
a_ZZ|\psi\rangle +
a_{Y}Y|\psi\rangle
\end{equation}
\tag{4}
$$

By also noting that the Pauli $Y$-matrix is equivalent, given the condition $Y=iXZ$, to the product $XZ$, we can rewrite the equation $(4)$ as follows

$$
\begin{equation}
U(\delta\theta, \delta\phi)|\phi\rangle =a_II|\psi\rangle + a_XX|\psi\rangle +
a_ZZ|\psi\rangle +
a_{XZ}XZ|\psi\rangle
\end{equation}
\tag{5}
$$

The above expression shows that any coherent error process can be decomposed into a sum from the Pauli basis set.

A quantum error correction code must, thus, have the ability to correct any coherent error. To be able to do so, the error correction process includes a projective measurement which causes a collapse of the superposition described above to a subset of its terms. It’s therefore possible to correct any error by being able to correct the errors described by the Pauli matrices.

## Error types

There are two fundamental quantum error types that need to be accounted for by quantum codes.

### Bit-flip

Pauli $X$-type errors can be thought of as quantum bit-flips that map $X|0\rangle = |1\rangle$ and $X|1\rangle = |0\rangle$. Thus, the action of an $X$-error on the general qubit state is

$$
\begin{equation}
X|\psi\rangle = \alpha X|0\rangle + \beta X|1\rangle = \alpha|1\rangle + \beta|0\rangle\,.
\end{equation}
\tag{6}
$$

### Phase-flip

The second quantum error type, the $Z$-error, is often referred to as a phase-flip. Phase-flips map the qubit basis states $Z|0\rangle= |0\rangle$ and $Z|1\rangle = -|1\rangle$ and therefore have the following action on the general qubit state

$$
\begin{equation}
Z|\psi\rangle = \alpha Z|0\rangle + \beta Z|1\rangle = \alpha|0\rangle - \beta|1\rangle\,.
\end{equation}
\tag{7}
$$

## Challenges of quantum error correction

From what we stated above about the digitization of quantum errors, it follows that it’s possible to reuse certain techniques from classical coding theory in quantum error correction. However, there still remain complications that don’t allow a straightforward translation of classical codes to quantum codes.

The first complication is posed by the no-cloning theorem for quantum states, the second complication arises from the susceptibility of qubits to both bit-flips and phase-flips errors, the final complication, specific to quantum errors, is raised by the wave-function collapse.

The process of quantum error correction must, therefore, add redundancy to the system in alternative ways and must be designed with the ability to detect both error types simultaneously, with the measurement operators carefully chosen so as not to cause the wave-function to collapse and erase the encoded information.

# Quantum codes redundancy & stabilizer measurements

Any quantum error correction code involves an encoding phase, an error-prone computing phase, a decoding phase and a correction phase.

The encoding process of one qubit into $n$ qubits is a representation of the logical states $|0_L\rangle$ and $|1_L\rangle$ as entangled states in the $n$-particle Hilbert space. Mathematically, we have

$$
\begin{equation}
|0_L\rangle = \sum_{i=0}^{2^n - 1}{\mu_i|i\rangle}\,;\,|1_L\rangle = \sum_{i=0}^{2^n - 1}{v_i|i\rangle}.
\end{equation}
\tag{8}
$$

In quantum codes, redundancy is added by expanding the Hilbert space in which the qubits are encoded so that the previous equation satisfies certain conditions.

## Two-qubit bit-flip code

We now analyze the two-qubit bit-flip code: a prototypical quantum code designed to detect a single-bit flip error.

The encoding stage of the two-qubit bit-flip code, acting on the general state $|\psi\rangle$ has the following action

$$
\begin{equation}
|\psi\rangle = \alpha|0\rangle + \beta|1\rangle\overset{\text{two-qubit bit-flip encoder}}{\longrightarrow}|\psi\rangle_L = \alpha|00\rangle + \beta|11\rangle =
\alpha|0\rangle_L + \beta|1\rangle_L
\end{equation}
\tag{9}
$$

note that this does not correspond to cloning the state.

By distributing the quantum information stored in the initial state $|\psi\rangle$ across the entangled two-qubit logical state $|\psi\rangle_L$, redundancy is introduced which can be later utilized by the process of error detection.

Before the encoding process takes place, the single qubit is found within a two-dimensional Hilbert space $|\psi\rangle \in \mathcal{H}_2 = \text{span}\{|0\rangle, |1\rangle\}$.

After the encoding operation, the logical qubit occupies a four-dimensional Hilbert space

$$
\begin{equation}
|\psi\rangle_L \in \mathcal{H}_4 = \text{span}\{|00\rangle, |01\rangle, |10\rangle, |11\rangle\}
\end{equation}
\tag{10}
$$

More specifically, the logical qubit is defined within a two-dimensional subspace of this expanded Hilbert space

$$
\begin{equation}
|\psi\rangle_L \in \mathcal{C} = \text{span}\{|00\rangle, |11\rangle\}
\end{equation}
\tag{11}
$$

where $\mathcal{C}$ is called the codespace.

Now, imagine that the logical qubit is subject to a bit-flip error on the first qubit resulting in the state

$$
\begin{equation}
X_1|\psi\rangle_L = \alpha|10\rangle + \beta|01\rangle
\end{equation}
\tag{12}
$$

where $X_1$ is a bit-flip error acting on the first qubit.

The resultant state is rotated into a new subspace

$$
\begin{equation}
X_1|\psi\rangle_L \in \mathcal{F} \subset \mathcal{H}_4
\end{equation}
\tag{13}
$$

where we call $\mathcal{F}$ the error subspace.

We can observe how $X_1-$ and $X_2$-errors rotate the logical state into the $\mathcal{F}$ subspace, therefore if the logical state $|\psi\rangle_L$ remains uncorrupted, it will occupy the codespace $\mathcal{C}$.

In order to distinguish which of the two sub-spaces the qubit occupies, a projective measurement is performed, which is a type of measurement that doesn’t compromise the encoded quantum information, therefore leaving the logical qubit unchanged. Measurements of this type are called stabilizer measurements and are crucial in the context of quantum coding. Distinction between the two sub-spaces is made possible by the fact that the $\mathcal{C}$ and $\mathcal{F}$ sub-spaces are mutually orthogonal.

A projective measurement of the form $Z_1Z_2$ is thus performed, whose $Z_1Z_2$ operator yields a $(+1)$ eigenvalue when applied to the following logical state

$$
\begin{equation}
Z_1Z_2|\psi\rangle_L = Z_1Z_2(\alpha|00\rangle + \beta|11\rangle) = (+1)|\psi\rangle_L\,.
\end{equation}
\tag{14}
$$

The $Z_1Z_2$ operator is said to stabilize the logical qubit $|\psi\rangle_L$ as it leaves it unchanged. Vice versa, the $Z_1Z_2$ operator projects the errored states $X_1|\psi\rangle_L$ and $X_2|\psi\rangle_L$ onto the $(-1)$ eigenspace. An ancilla qubit $|0\rangle_A$ is used to perform the measurement of the $Z_1Z_2$ stabilizer. We call this stage syndrome extraction step and it transforms the quantum state as follows

$$
\begin{equation}
E|\psi\rangle_L|0\rangle_A \overset{\text{syndrome extraction}}\longrightarrow
\frac{1}{2}(I_1I_2 + Z_1Z_2)E|\psi\rangle_L|0\rangle_A
+ \frac{1}{2}(I_1I_2 - Z_1Z_2)E|\psi\rangle_L|1\rangle_A
\end{equation}
\tag{15}
$$

where $E$ is an error from the set $\{I, X_1, X_2, X_1X_2\}$.

Consider the case where $E=X_1$ so that the logical state occupies the error space $\mathcal{F}$. In this scenario, it can be seen that the first term in the equation $(15)$ goes to zero. The ancilla qubit is therefore measured deterministically as $1$. Considering the other error patterns, we see that if the logical state is in the codespace (i.e., if $E=\{I, X_1X_2\}$) then the ancilla is measured as $0$. Likewise, if the logical state is in the error subspace (i.e., if $E=\{X_1,X_2\}$) then the ancilla is measured as $1$. The outcome of the ancilla qubit measurement is referred to as a syndrome and tells us whether or not the logical state has been subject to an error.

Before practicing what we presented, we build two helper functions: one that injects a random error in a random qubit in the circuit and one that runs the circuit and return the results.

In [ ]:
from enum import Enum

class ErrorType(Enum):
  BitFlip = "X"
  PhaseFlip = "Z"


def inject_error(circuit: QuantumCircuit, num_qubits: int, error_type: ErrorType = None, qubit_num: int = None) -> None:
  """Injects an error to a qubit.

  Args:
    circuit (QuantumCircuit): A quantum circuit.
    num_qubits (int): The number of qubits that can error.
    error_type (ErrorType, optional): The type of error to be injected, randomly generated if not provided.
    qubit_num (int, optional): The qubit to inject the error in, randomly generated if not provided.
  """
  if (qubit_num is None):
    qubit_num = randint(0, num_qubits - 1)
  
  if (error_type is None):
    error_type = ErrorType.BitFlip if randint(0, 1) == 0 else ErrorType.PhaseFlip
  
  if (error_type == ErrorType.BitFlip):
    circuit.x(qubit_num)
  elif (error_type == ErrorType.PhaseFlip):
    circuit.z(qubit_num)
  
  circuit.barrier()
  
  print(f"Injected {error_type.value} error on qubit {qubit_num + 1}")

def run_and_measure(circuit: QuantumCircuit, shots: int = 1024) -> dict:
  """Compiles and runs the circuit with the Aer Simulator backend then returns the results.

  Args:
    circuit (QuantumCircuit): A Quantum Circuit.
    shots (int, optional): The number of times the circuit has to be executed. Defaults to 1000.

  Returns:
    Dict: A dictonary representing the results.
  """
  simulator = Aer.get_backend('aer_simulator')

  compiled_circuit = transpile(circuit, simulator)

  # Execute the circuit on the qasm simulator
  job = simulator.run(compiled_circuit, shots=shots)

  # Grab results from the job
  result = job.result()
  cnt = result.get_counts(compiled_circuit)
  
  return cnt

We can now move to the implementation of the first circuit.

In [ ]:
def encode_two_qubit_bitflip() -> QuantumCircuit:
  """Builds and returns the encoding circuit for the two-qubit bit-flip code.

  Returns:
    QuantumCircuit: Circuit with 2 data qubits and 1 ancilla.
  """
  qreg_q = QuantumRegister(3, 'q')
  creg_c = ClassicalRegister(1, 'c')
  circuit = QuantumCircuit(qreg_q, creg_c)

  circuit.cx(qreg_q[0], qreg_q[1])
  
  circuit.barrier()

  return circuit


def extract_syndrome_two_qubit_bitflip(circuit: QuantumCircuit) -> None:
  """Extracts the bit-flip error syndrome using the ancilla."""
  circuit.cx(0, 2)
  circuit.cx(1, 2)
  circuit.measure(2, 0)
  
  circuit.barrier()


circuit = encode_two_qubit_bitflip()
inject_error(circuit, 2, ErrorType.BitFlip)
extract_syndrome_two_qubit_bitflip(circuit)

result = run_and_measure(circuit)

print(result)

We can observe how a bit-flip error is detected if the ancilla qubit is measured to be $1$, whereas the qubit remained uncorrupted (or in the worst case, both qubits errored) if the ancilla qubit is measured to be $0$.

## Two-qubit phase-flip code

We now analyze the two-qubit phase-flip code: a prototypical quantum code designed to detect a single-phase flip error, which represents the counterpart of the two-qubit bit-flip code.

This code shares some similarities with the code previously introduced, the key intuition behind a phase-flip error detection is switching to the Hadamard basis.

Therefore, the encoding stage of the two-qubit phase-flip code, acting on the general state $|\psi\rangle$ has the following action

$$
\begin{equation}
|\psi\rangle = \alpha|0\rangle + \beta|1\rangle\overset{\text{two-qubit phase-flip encoder}}{\longrightarrow}|\psi\rangle_L = \alpha|++\rangle + \beta|--\rangle =
\alpha|+\rangle_L + \beta|-\rangle_L
\end{equation}
\tag{16}
$$

where

$$
\begin{equation}
|\pm\rangle = \frac{|0\rangle \pm |1\rangle}{\sqrt2}
\end{equation}
\tag{17}
$$

In the Hadamard basis, the expanded Hilbert space is defined as follows

$$
\begin{equation}
\mathcal{H}_4 = \text{span}\{|++\rangle, |+-\rangle, |-+\rangle, |--\rangle\}
\end{equation}
\tag{18}
$$

while code- and error spaces are defined as follows

$$
\begin{equation}
\mathcal{C} = \text{span}\{|++\rangle, |--\rangle\}
\in \mathcal{H}_4
\end{equation}
\tag{19}
$$

$$
\begin{equation}
\mathcal{F} = \text{span}\{|+-\rangle, |-+\rangle\}
\in \mathcal{H}_4
\end{equation}
\tag{20}
$$

The action of an error $E \in \{Z_1, Z_2\}$ rotates the logical state into the $\mathcal{F}$ subspace, whereas, if $E \in \{I, Z_1Z_2\}$ the logical state occupies the $\mathcal{C}$ subspace.

As previously stated for the two-qubit bit-flip code, the sub-spaces $\mathcal{C}$ and $\mathcal{F}$ are mutually orthogonal, thus it is possible to distinguish which subspace the logical qubit occupies.

In the case of a phase-flip error, the state can be differentiated between the two sub-spaces via a projective measurement of the form $X_1X_2$. The $X_1X_2$ operator yields a $(+1)$ eigenvalue when applied to the logical state

$$
\begin{equation}
X_1X_2|\psi\rangle_L = X_1X_2(\alpha|++\rangle + \beta|--\rangle) = (+1)|\psi\rangle_L
\end{equation}
\tag{21}
$$

Conversely, the $X_1X_2$ operator yields a $(-1)$ eigenvalue when applied to the errored states $Z_1|\psi\rangle_L$ and $Z_2|\psi\rangle_L$.

The syndrome extraction step transforms the quantum state as follows

$$
\begin{equation}
E|\psi\rangle_L|0\rangle_A \overset{\text{syndrome extraction}}\longrightarrow
\frac{1}{2}(I_1I_2 + X_1X_2)E|\psi\rangle_L|0\rangle_A
+ \frac{1}{2}(I_1I_2 - X_1X_2)E|\psi\rangle_L|1\rangle_A
\end{equation}
\tag{22}
$$

where $E$ is an error from the set $\{I, Z_1, Z_2, Z_1Z_2\}$.

We now investigate this code in practice.

In [ ]:
def encode_two_qubit_phaseflip() -> QuantumCircuit:
  """Builds and returns the encoding circuit for the two-qubit phase-flip code.

  Returns:
    QuantumCircuit: Circuit with 2 data qubits and 1 ancilla.
  """
  qreg_q = QuantumRegister(3, 'q')
  creg_c = ClassicalRegister(1, 'c')
  circuit = QuantumCircuit(qreg_q, creg_c)

  circuit.h(qreg_q[0])
  circuit.h(qreg_q[1])
  circuit.cx(qreg_q[0], qreg_q[1])

  circuit.barrier()
  return circuit


def extract_syndrome_two_qubit_phaseflip(circuit: QuantumCircuit) -> None:
  """Extracts the phase-flip error syndrome using the ancilla."""
  circuit.h(0)
  circuit.h(1)

  circuit.h(2)
  circuit.cz(0, 2)
  circuit.cz(1, 2)
  circuit.h(2)

  circuit.measure(2, 0)
  circuit.barrier()


def decode_two_qubit_phaseflip(circuit: QuantumCircuit) -> None:
  """Decodes by rotating logical qubits back to computational basis."""
  circuit.h(0)
  circuit.h(1)
  circuit.barrier()


circuit = encode_two_qubit_phaseflip()
inject_error(circuit, 2, ErrorType.PhaseFlip)
extract_syndrome_two_qubit_phaseflip(circuit)
decode_two_qubit_phaseflip(circuit)

result = run_and_measure(circuit)

print(result)

We can observe how a phase-flip error is detected if the ancilla qubit is measured to be $1$, whereas the qubit remained uncorrupted (or in the worst case, both qubits errored) if the ancilla qubit is measured to be $0$.

## The three-qubit error correction code

The two-qubit codes previously analyzed inform us of the presence of an error, but do not provide enough information to allow us to infer which qubit the error occurred on, it is only possible to detect whether an error has occurred. The reason behind this is the size of the redundancy added to the system, which is inadequate for a consistent correction process.

In order to create an error correction code with the ability to both detect and localize errors, multiple stabilizer measurements need to be performed.

### Three-qubit bit-flip error correction code

In this section we describe the bit-flip three-qubit code, the natural extension of the two-qubit bit-flip code designed to detect and correct a single bit-flip error.

The encoding operation distributes the quantum information across an entangled three-party state to give a logical state of the form $|\psi\rangle_L = \alpha|000\rangle + \beta|111\rangle$.
This logical state occupies an eight-dimensional Hilbert space that can be partitioned into four two-dimensional sub-spaces as follows

$$
\begin{equation}
\begin{split}
\mathcal{C} = \text{span}\{|000\rangle, |111\rangle\},\\
\qquad \mathcal{F_1} = \text{span}\{|100\rangle, |011\rangle\},\\ \mathcal{F}_2 = \text{span}\{|010\rangle, |101\rangle\},\\ \mathcal{F_3} = \text{span}\{|001\rangle, |110\rangle\}
\end{split}
\end{equation}
\tag{23}
$$

where $\mathcal{C}$ is the logical code space and $F_{\{1,2,3\}}$ are the logical error spaces. We see that each single qubit error from the set $E=\{X_1, X_2, X_3\}$ will rotate the codespace to a unique error space so that $X_i|\psi\rangle_L \in \mathcal{F}_i$.

In order to differentiate between these sub-spaces, we perform two stabilizer measurements, $Z_1Z_2$ and $Z_2Z_3$ via the circuit shown in the figure. The resultant syndrome table for the single-qubit errors is given in the table.

| Error | Syndrome | Error | Syndrome |
| --- | --- | --- | --- |
| $I_1I_2I_3$ | $00$ | $X_1X_2I_3$ | $01$ |
| $X_1I_2I_3$ | $10$ | $I_1X_2X_3$ | $10$ |
| $I_1X_2I_3$ | $11$ | $X_1I_2X_3$ | $11$ |
| $I_1I_2X_3$ | $01$ | $X_1X_2X_3$ | $00$ |

![Three-qubit bit-flip code](./3bftqc.jpg)
The circuit diagram of the bit-flip three-qubit code. Stages are separated by barriers. The first stage depicts the encoding gates. The second stage depicts the syndrome extraction gates. The third gates depicts the ancillae qubits measurement. Finally, the last stage depicts the error correction process.

We now analyze the code in practice.

In [ ]:
def encode_three_qubit_bitflip() -> QuantumCircuit:
  """Builds and returns the encoding circuit for the three-qubit bit-flip code.

  Returns:
    QuantumCircuit: Circuit with 3 data qubits, 2 ancillae, 3 data classical bits and 2 syndrome classical bits.
  """
  qreg_q = QuantumRegister(5, 'q')
  creg_c = ClassicalRegister(3, 'c')
  creg_s = ClassicalRegister(2, 's')
  circuit = QuantumCircuit(qreg_q, creg_c, creg_s)

  circuit.cx(qreg_q[0], qreg_q[1])
  circuit.cx(qreg_q[0], qreg_q[2])
  circuit.barrier()

  return circuit


def extract_syndrome_three_qubit_bitflip(circuit: QuantumCircuit):
  """Extracts the bit-flip error syndrome using the ancillae."""
  circuit.cx(0, 3)
  circuit.cx(1, 3)
  circuit.cx(1, 4)
  circuit.cx(2, 4)
  circuit.barrier()

  circuit.measure(3, 4)
  circuit.measure(4, 3)
  circuit.barrier()


def correct_error_three_qubit_bitflip(circuit: QuantumCircuit):
  """Corrects the bit-flip detected error."""
  with circuit.if_test((4, 1)):
    with circuit.if_test((3, 0)):
      circuit.x(0)

  with circuit.if_test((4, 1)):
    with circuit.if_test((3, 1)):
      circuit.x(1)

  with circuit.if_test((4, 0)):
    with circuit.if_test((3, 1)):
      circuit.x(2)


def decode_three_qubit_bitflip(circuit: QuantumCircuit):
  """Applies a bit-flip decoding operation"""
  circuit.measure(0, 0)
  circuit.measure(1, 1)
  circuit.measure(2, 2)


circuit = encode_three_qubit_bitflip()
inject_error(circuit, 3, ErrorType.BitFlip)
extract_syndrome_three_qubit_bitflip(circuit)
correct_error_three_qubit_bitflip(circuit)
decode_three_qubit_bitflip(circuit)

result = run_and_measure(circuit)

print(result)

As we can see from the results, we’re able to infer which qubit has errored and thus apply a suitable recovery process.

### Phase-flip three-qubit error correction code

In this section we describe the phase-flip three-qubit code in a similar fashion, the natural extension of the two-qubit bit-flip code designed to detect and correct a single phase-flip error.

The encoding operation distributes the quantum information across an entangled three-party state to give a logical state of the form $|\psi\rangle_L = \alpha|+++\rangle + \beta|---\rangle$.
This logical state occupies an eight-dimensional Hilbert space that can be partitioned into four two-dimensional sub-spaces as follows

$$
\begin{equation}
\begin{split}
\mathcal{C} = \text{span}\{|+++\rangle, |---\rangle\},\\
\mathcal{F_1} = \text{span}\{|-++\rangle, |+--\rangle\},\\
\mathcal{F}_2 = \text{span}\{|+-+\rangle, |-+-\rangle\},\\
\mathcal{F_3} = \text{span}\{|++-\rangle, |--+\rangle\}
\end{split}
\end{equation}
\tag{24}
$$

where $\mathcal{C}$ is the logical code space and $F_{\{1,2,3\}}$ are the logical error spaces. We see that each single qubit error from the set $E=\{Z_1, Z_2, Z_3\}$ will rotate the codespace to a unique error space so that $Z_i|\psi\rangle_L \in \mathcal{F}_i$.

In order to differentiate between these sub-spaces, we perform two stabilizer measurements, $X_1X_2$ and $X_2X_3$ via the circuit shown in the figure. The resultant syndrome table for the single-qubit errors is given in the table.

| Error | Syndrome | Error | Syndrome |
| --- | --- | --- | --- |
| $I_1I_2I_3$ | $00$ | $Z_1Z_2I_3$ | $01$ |
| $Z_1I_2I_3$ | $10$ | $I_1Z_2Z_3$ | $10$ |
| $I_1Z_2I_3$ | $11$ | $Z_1I_2Z_3$ | $11$ |
| $I_1I_2Z_3$ | $01$ | $Z_1Z_2Z_3$ | $00$ |

![Three-qubit phase-flip code](./3pftqc.jpg)
The circuit diagram of the phase-flip three-qubit code. Stages are separated by barriers. The first stage depicts the encoding gates. The second stage depicts the syndrome extraction gates. The third gates depicts the ancillae qubits measurement. Finally, the last stage depicts the error correction process.

We begin the application.

In [ ]:
def encode_three_qubit_phaseflip() -> QuantumCircuit:
  """Builds and returns the encoding circuit for the three-qubit phase-flip code.

  Returns:
    QuantumCircuit: Circuit with 3 data qubits, 2 ancillae, 3 data classical bits and 2 syndrome classical bits.
  """
  qreg_q = QuantumRegister(5, 'q')
  creg_c = ClassicalRegister(3, 'c')
  creg_s = ClassicalRegister(2, 's')
  circuit = QuantumCircuit(qreg_q, creg_c, creg_s)

  circuit.h(qreg_q[0])
  circuit.h(qreg_q[1])
  circuit.h(qreg_q[2])
  circuit.cx(qreg_q[0], qreg_q[1])
  circuit.cx(qreg_q[0], qreg_q[2])
  circuit.barrier()

  return circuit


def extract_syndrome_three_qubit_phaseflip(circuit: QuantumCircuit):
  """Extracts the phase-flip error syndrome using the ancillae."""
  circuit.h(0)
  circuit.h(1)
  circuit.cx(0, 3)
  circuit.cx(1, 3)
  circuit.h(0)
  circuit.h(1)

  circuit.h(1)
  circuit.h(2)
  circuit.cx(1, 4)
  circuit.cx(2, 4)
  circuit.h(1)
  circuit.h(2)

  circuit.barrier()

  circuit.measure(3, 4)
  circuit.measure(4, 3)
  circuit.barrier()


def correct_error_three_qubit_phaseflip(circuit: QuantumCircuit):
  """Corrects the detected phase-flip error."""
  with circuit.if_test((4, 1)):
    with circuit.if_test((3, 0)):
      circuit.z(0)

  with circuit.if_test((4, 1)):
    with circuit.if_test((3, 1)):
      circuit.z(1)

  with circuit.if_test((4, 0)):
    with circuit.if_test((3, 1)):
      circuit.z(2)
  
  circuit.barrier()


def decode_three_qubit_phaseflip(circuit: QuantumCircuit):
  """Applies a phase-flip decoding operation"""
  circuit.h(0)
  circuit.h(1)
  circuit.h(2)

  circuit.measure(0, 0)
  circuit.measure(1, 1)
  circuit.measure(2, 2)


circuit = encode_three_qubit_phaseflip()
inject_error(circuit, 3, ErrorType.PhaseFlip)
extract_syndrome_three_qubit_phaseflip(circuit)
correct_error_three_qubit_phaseflip(circuit)
decode_three_qubit_phaseflip(circuit)

result = run_and_measure(circuit)

print(result)

As we’ve seen previously, we’re able to correctly infer which qubit has errored and thus apply the correct recovery operation.

## Quantum code distance

As for classical codes, the distance of a quantum code is defined as the minimum number of errors that will go undetected. For the three-qubit codes previously described, we could conclude that they both have distance $d=3$, this would be the case if the qubits were only susceptible to $X$- and $Z$-errors separately. The three-qubit bit-flip error correction code can’t detect phase-flips errors and, likewise, the three-qubit phase-flip error correction code can’t detect bit-flips errors. As qubits are susceptible to both error types at the same time, each of the correction codes described in the previous sections have distance $d=1$.

To overcome the limitations of the simple three-qubit codes, we turn to the stabilizer formalism, which generalizes the encoding process and allows for the systematic construction of codes with higher distance and the ability to correct multiple error types simultaneously.

# Stabilizer codes

We now introduce the theoretical basics of the stabilizer codes.

A register of $k$ data qubits, $|\psi\rangle_D$, is entangled with $m=n-k$ redundancy qubits $|0\rangle_R$ via an encoding operation to create a logical qubit $|\psi\rangle_L$. At this stage, the data previously stored solely in $|\psi\rangle_D$ is distributed across the expanded Hilbert space.

Errors can then be detected by performing $m$ stabilizer measurements $P_i$.

For each stabilizer $P_i$, the syndrome extraction circuit maps the logical state as follows

$$
\begin{align}
E|\psi\rangle_L|0\rangle_A \overset{\text{syndrome extraction}}\longrightarrow
\frac{1}{2}(I^{\otimes n} +P_i)E|\psi\rangle_L|0\rangle_A
+ \frac{1}{2}(I^{\otimes n} - P_i)E|\psi\rangle_L|1\rangle_A
\end{align}
\tag{25}
$$

We can observe how, if the stabilizer $P_i$ commutes with an error $E$, the measurement of the ancilla qubit deterministically returns $0$. Whereas, if the stabilizer $P_i$ anti-commutes with an error $E$, the measurement returns $1$. The task of constructing a stabilizer code involves finding stabilizers $P$ that commute and anti-commute with an error $E$ when deemed necessary.

The measurements result in an $m$-bit syndrome, which, for a well designed code, allows us to understand the error occurred and, thus, infer the correct and appropriate recovery option.

In this way, the stabilizer code formalism provides a compact and powerful language for describing and analyzing a wide range of quantum error-correcting codes, including those capable of simultaneously correcting $X$-, $Z$-, and combined errors. The simple three-qubit codes discussed earlier can be seen as elementary examples of this framework, while more complex codes—such as the nine-qubit Shor code and the five-qubit code by Laflamme et al.—fully exploit this structure to achieve greater distances and more robust error-correction capabilities.

# Shor $[[9, 1, 3]]$ code

We now present the Shor nine-qubit code and the technique it is based on, known as code concatenation. The Shor nine-qubit code allows the detection and correction of both types of errors on a logical qubit meaning it corrects arbitrary single-qubit errors, using the stabilizer formalism previously introduced.

Code concatenation entails combining multiple simpler codes in a hierarchical manner to produce a more robust code by feeding the output of one code into the input of another.

The Shor nine-qubit code is constructed by concatenating two codes: the three-qubit bit-flip code and the three-qubit phase-flip code.

The encoding stages and stabilizers of the three-qubit codes previously introduced are defined as follows

$$
\begin{equation}
\mathcal{C}_{3\text{b}} = \text{span}\{|0\rangle_{3\text{b}} = |000\rangle, |1\rangle_{3\text{b}} = |111\rangle\},\quad \mathcal{S}_{3\text{b}} = \langle Z_1Z_2, Z_2Z_3\rangle
\end{equation}
\tag{26}
$$

$$
\begin{equation}
\mathcal{C}_{3\text{p}} = \text{span}\{|0\rangle_{3\text{p}} = |+++\rangle, |1\rangle_{3\text{p}} = |---\rangle\},\quad \mathcal{S}_{3\text{p}} = \langle X_1X_2, X_2X_3\rangle
\end{equation}
\tag{27}
$$

Embedding the bit-flip code into the code-words of the phase-flip code results in the creation of the nine-qubit code. The $|0\rangle_{3p}$ code-word of the phase-flip code is thus mapped to a nine-qubit code-word $|0\rangle_9$ as follows

$$
\begin{equation}
|0\rangle_{3\text{p}} = |+++\rangle\overset{\text{concatenation}}\longrightarrow|0\rangle_9 = |+\rangle_{3\text{b}}|+\rangle_{3\text{b}}|+\rangle_{3\text{b}}
\end{equation}
\tag{28}
$$

where $|+\rangle_{3b} = \frac{1}{\sqrt2}(|000\rangle + |111\rangle)$ is a logical state of the bit-flip code.

Similarly, the $|1\rangle_{3p}$ code-word of the phase-flip code is mapped to a nine-qubit code-word $|1\rangle_9$ as follows

$$
\begin{equation}
|1\rangle_{3\text{p}} = |---\rangle\overset{\text{concatenation}}\longrightarrow|1\rangle_9 = |-\rangle_{3\text{b}}|-\rangle_{3\text{b}}|-\rangle_{3\text{b}}
\end{equation}
\tag{29}
$$

where $|-\rangle_{3b} = \frac{1}{\sqrt2}(|000\rangle - |111\rangle)$.

We can obtain the codespace for the Shor code using the previous definitions

$$
\begin{equation}
\mathcal{C}_{[[9, 1, 3]]} = \text{span}
\begin{cases}
|0\rangle_9 = \frac{1}{\sqrt2}(|000\rangle + |111\rangle)(|000\rangle + |111\rangle)(|000\rangle + |111\rangle)\\
|1\rangle_9 = \frac{1}{\sqrt2}(|000\rangle - |111\rangle)(|000\rangle - |111\rangle)(|000\rangle - |111\rangle)
\end{cases}
\end{equation}
\tag{30}
$$

and the stabilizers are given by

$$
\begin{equation}
\begin{split}
S_{[[9, 1, 3]]} = \langle Z_1 Z_2, Z_2 Z_3, Z_4 Z_5, Z_5 Z_6, Z_7 Z_8, Z_8Z_9,\\
X_1X_2X_3X_4X_5X_6, X_4X_5X_6X_7X_8X_9
\rangle
\end{split}
\end{equation}
\tag{31}
$$

Before moving on to the implementation, we write the syndrome table for this code.

| Error | Syndrome | Error | Syndrome |
| --- | --- | --- | --- |
| $X_1$ | $10000000$ | $Z_1$ | $00000010$ |
| $X_2$ | $11000000$ | $Z_2$ | $00000010$ |
| $X_3$ | $01000000$ | $Z_3$ | $00000010$ |
| $X_4$ | $00100000$ | $Z_4$ | $00000011$ |
| $X_5$ | $00110000$ | $Z_5$ | $00000011$ |
| $X_6$ | $00010000$ | $Z_6$ | $00000011$ |
| $X_7$ | $00001000$ | $Z_7$ | $00000001$ |
| $X_8$ | $00001100$ | $Z_8$ | $00000001$ |
| $X_9$ | $00000100$ | $Z_9$ | $00000001$ |

We can observe how each one of the $X$-errors that occurs has a different syndrome, while $Z$-errors that occur in the same block of the code have the same syndrome. Naturally, this degeneracy seems to pose a problem, however it does not reduce the code distance. To see why this is the case, consider the single-qubit errors $Z_4$ and $Z_5$ for which the syndrome is the same, thus the decoder doesn’t possess sufficient information to differentiate between the two errors and will apply the same recovery operation in either case. We will assume that the recovery operation output is $\mathcal{R} = Z_4$. Now, for the case where $E=Z_4$ the recovery operation restores the logical state correctly. For the case where $E= Z_2$, the recovery operation applies the operation $\mathcal{R}E|\psi\rangle_9 = Z_4Z_5|\psi\rangle_9 =|\psi\rangle_9$ and this is true because is a stabilizer of $\mathcal{S}_{[[9, 1, 3]]}$.

![The encoding circuit of the nine-qubit code.](./scec.jpg)

The encoding circuit of the nine-qubit code.

We now move to the implementation of this code by first building the various circuits.

In [3]:
def initialize_shor_circuit() -> QuantumCircuit:
  """Initializes the circuit utilized for Shor encoding.

  Returns:
      QuantumCircuit: A circuit containing 17 qubits and 17 classical bits.
  """
  qreg_q = QuantumRegister(17, 'q')
  creg_s = ClassicalRegister(8, 's')
  creg_l = ClassicalRegister(9, 'l')
  circuit = QuantumCircuit(qreg_q, creg_s, creg_l)

  return circuit


def encode_shor(circuit: QuantumCircuit) -> QuantumCircuit:
  """Encodes a circuit with the Shor code.

  Args:
      circuit (QuantumCircuit): The circuit containing 17 qubits and 17 classical bits.

  Returns:
      QuantumCircuit: The encoded circuit.
  """
  circuit.cx(0, 3)
  circuit.cx(0, 6)
  circuit.h(0)
  circuit.h(3)
  circuit.h(6)
  circuit.cx(0, 1)
  circuit.cx(0, 2)
  circuit.cx(3, 4)
  circuit.cx(3, 5)
  circuit.cx(6, 7)
  circuit.cx(6, 8)

  return circuit


def extract_syndrome_shor_bitflip(circuit: QuantumCircuit):
  """Adds the Shor syndrome extraction for a bit-flip error.

  Args:
      circuit (QuantumCircuit): The circuit to add the syndrome extraction in.
  """
  circuit.cx(0, 9)
  circuit.cx(1, 9)

  circuit.cx(1, 10)
  circuit.cx(2, 10)

  circuit.cx(3, 11)
  circuit.cx(4, 11)

  circuit.cx(4, 12)
  circuit.cx(5, 12)

  circuit.cx(6, 13)
  circuit.cx(7, 13)

  circuit.cx(7, 14)
  circuit.cx(8, 14)

  circuit.barrier()


def measure_syndrome_shor_bitflip(circuit: QuantumCircuit):
  """Adds the bit-flip syndrome measurements to a Shor circuit. 

  Args:
      circuit (QuantumCircuit): The circuit to add the measurements in.
  """
  circuit.measure(9, 7)
  circuit.measure(10, 6)
  circuit.measure(11, 5)
  circuit.measure(12, 4)
  circuit.measure(13, 3)
  circuit.measure(14, 2)

  circuit.barrier()


def correct_error_shor_bitflip(circuit: QuantumCircuit):
  """Adds the Shor error correction for a bit-flip error.

  Args:
      circuit (QuantumCircuit): The circuit to add the error correction in.
  """
  # X1 error
  circuit.x(10)
  circuit.ccx(9, 10, 0)
  circuit.x(10)
  
  # X2 error
  circuit.ccx(9, 10, 1)
  
  # X3 error
  circuit.x(9)
  circuit.ccx(9, 10, 2)
  circuit.x(9)
  
  # X4 error
  circuit.x(12)
  circuit.ccx(11, 12, 3)
  circuit.x(12)
  
  # X5 error
  circuit.ccx(11, 12, 4)
  
  # X6 error
  circuit.x(11)
  circuit.ccx(11, 12, 5)
  circuit.x(11)
  
  # X7 error
  circuit.x(14)
  circuit.ccx(13, 14, 6)
  circuit.x(14)
  
  # X8 error
  circuit.ccx(13, 14, 7)
  
  # X9 error
  circuit.x(13)
  circuit.ccx(13, 14, 8)
  circuit.x(13)

  circuit.barrier()


def extract_syndrome_shor_phaseflip(circuit: QuantumCircuit):
  """Adds the Shor syndrome extraction for a phase-flip error.

  Args:
      circuit (QuantumCircuit): The circuit to add the syndrome extraction in.
  """
  for i in range(6):
    circuit.h(i)
  circuit.cx(0, 15)
  circuit.cx(1, 15)
  circuit.cx(2, 15)
  circuit.cx(3, 15)
  circuit.cx(4, 15)
  circuit.cx(5, 15)
  for i in range(6):
    circuit.h(i)

  for i in range(3, 9):
    circuit.h(i)
  circuit.cx(3, 16)
  circuit.cx(4, 16)
  circuit.cx(5, 16)
  circuit.cx(6, 16)
  circuit.cx(7, 16)
  circuit.cx(8, 16)
  for i in range(3, 9):
    circuit.h(i)

  circuit.barrier()


def measure_syndrome_shor_phaseflip(circuit: QuantumCircuit):
  """Adds the phase-flip syndrome measurements to a Shor circuit. 

  Args:
      circuit (QuantumCircuit): The circuit to add the measurements in.
  """
  circuit.measure(15, 1)
  circuit.measure(16, 0)

  circuit.barrier()


def correct_error_shor_phaseflip(circuit: QuantumCircuit):
  """Adds the Shor error correction for a bit-flip error.

  Args:
      circuit (QuantumCircuit): The circuit to add the error correction in.
  """
  #Z1-2-3 error
  circuit.x(16)
  circuit.ccz(15, 16, 0)
  circuit.x(16)
  
  #Z4-5-6 error
  circuit.ccz(15, 16, 3)
  
  #Z7-8-9 error
  circuit.x(15)
  circuit.ccz(15, 16, 6)
  circuit.x(15)

  circuit.barrier()


def decode_shor(circuit: QuantumCircuit):
  """Decodes a Shor code encoded circuit.

  Args:
      circuit (QuantumCircuit): The circuit to add the decoding in.
  """
  circuit.compose(encode_shor(
      initialize_shor_circuit()).inverse(), inplace=True)


def measure_qubits_shor(circuit: QuantumCircuit):
  """Measures the qubits encoded with the Shor code.

  Args:
      circuit (QuantumCircuit): The circuit to add the measurements in.
  """
  circuit.measure(0, 16)
  circuit.measure(1, 15)
  circuit.measure(2, 14)
  circuit.measure(3, 13)
  circuit.measure(4, 12)
  circuit.measure(5, 11)
  circuit.measure(6, 10)
  circuit.measure(7, 9)
  circuit.measure(8, 8)

  circuit.barrier()

We continue by investigating the errors that may take place and the suitable recovery operations to apply.

First, we’ll consider bit-flip errors result in a different syndrome depending on the qubit flipped. This allows us to choose the suitable recover operation, according to the table.

In [ ]:
circuit = initialize_shor_circuit()
encode_shor(circuit)
inject_error(circuit, 9, ErrorType.BitFlip)
extract_syndrome_shor_bitflip(circuit)
measure_syndrome_shor_bitflip(circuit)
correct_error_shor_bitflip(circuit)
decode_shor(circuit)
measure_qubits_shor(circuit)

result = run_and_measure(circuit, 1024)

print(result)

Secondly, we’ll analyze phase-flip errors. In this situation, the error can be corrected by applying a $Z$ gate to one of the qubits of the block, according to the syndrome.

In [ ]:
circuit = initialize_shor_circuit()
encode_shor(circuit)
inject_error(circuit, 9, ErrorType.PhaseFlip)
extract_syndrome_shor_phaseflip(circuit)
measure_syndrome_shor_phaseflip(circuit)
correct_error_shor_phaseflip(circuit)
decode_shor(circuit)
measure_qubits_shor(circuit)

result = run_and_measure(circuit, 1024)

print(result)

Finally, we’ll analyze the case in which both types of error occur. This code is able to detect and correct any single-qubit error, as introduced. In fact, the correction process for these two kind of errors can be applied independently and even without any regard for the order.

In [ ]:
circuit = initialize_shor_circuit()
circuit.h(0)
encode_shor(circuit)
inject_error(circuit, 9, ErrorType.BitFlip, qubit_num=0)
inject_error(circuit, 9, ErrorType.PhaseFlip, qubit_num=0)
extract_syndrome_shor_bitflip(circuit)
extract_syndrome_shor_phaseflip(circuit)
measure_syndrome_shor_bitflip(circuit)
measure_syndrome_shor_phaseflip(circuit)
correct_error_shor_bitflip(circuit)
correct_error_shor_phaseflip(circuit)
decode_shor(circuit)
measure_qubits_shor(circuit)

result = run_and_measure(circuit, 1024)

print(result)

# 5-qubit code

We now present the five-qubit error-correcting code, also known as the $[[5,1,3]]$ code, the smallest quantum error correcting code that can protect a logical qubit from any arbitrary single qubit error.

This code relies on the orthogonality property, which requires a different subspace for each of the three errors every qubit can suffer, plus another one for the logical state. This results in a total of $3n+1$ sub-spaces, in order to hold both logical states we have to double the number of sub-spaces, which results in $2(3n+1)$ sub-spaces.

A particular condition must be satisfied in order to have enough space and to assure errors act on orthogonal sub-spaces:

$$
\begin{equation}
2(3n+1)\leq2^n
\end{equation}
\tag{32}
$$

Shor’s $n=9$-code satisfies this constraint and the smallest possible number that fulfills the inequality is $n=5$.

The orthogonality conditions, assuming simplicity, can be written as follows

$$
\begin{equation}
\begin{split}
\sum_{k-\text{even},\,l-\text{even}}{|\mu_i|^2} = \sum_{k-\text{even},\,l-\text{odd}}{|\mu_i|^2} = \sum_{k-\text{odd},\,l-\text{even}}{|\mu_i|^2} =
\sum_{k-\text{odd},\,l-\text{odd}}{|\mu_i|^2}\\
k,l = 1, \dots, 5\qquad\qquad\qquad\qquad\qquad\qquad\qquad
\end{split}
\end{equation}
\tag{33}
$$

Where $\mu_i$ represents an algebraic coefficient which defines the encoding, note that a similar condition holds for the coefficient $v_i$, ensuring that both logical code-words respect orthogonality with reference to the equation $(8)$.

We can observe that the conditions conveyed in the previous equations are extremely restrictive, this allows to find what are the eight states $|i\rangle$ allowed in equation $(8)$ defining the support of the code. Thus, we can conclude that, in order for the property to be satisfied (assuming simplicity), with five qubits, at least eight states in superposition are needed.

$$
\begin{equation}
|0_L\rangle = |b_1\rangle|00\rangle - |b_3\rangle|11\rangle+|b_7\rangle|10\rangle + |b_5\rangle|01\rangle\\
|1_L\rangle = -|b_2\rangle|11\rangle - |b_4\rangle|00\rangle+|b_8\rangle|01\rangle - |b_6\rangle|10\rangle
\end{equation}
\tag{34}
$$

where the 3-particle Bell states are defined (up to normalization) as $|b_{1-2}\rangle = (|000\rangle \pm |111\rangle)$, $|b_{3-4}\rangle = (|100\rangle \pm |011\rangle)$, $|b_{5-6}\rangle = (|010\rangle \pm |101\rangle)$, $|b_{7-8}\rangle = (|110\rangle \pm |001\rangle)$.

Now, the notable feature of this method is that the circuit for the error correction phase is exactly the same as the encoding circuit, but run backwards, this significantly differs from the codes discussed earlier.
Note that the following syndrome table and circuit we analyze differ from the ones described in Laflamme et al.’s article, but they are equivalent from a conceptual and corrective standpoint.
Before implementing this code, we write the syndrome table.

| Error | Syndrome | Error | Syndrome | Error | Syndrome |
| --- | --- | --- | --- | --- | --- |
| $X_1$ | $0101$ | $Z_1$ | $1000$ | $X_1Z_1$ | $1101$ |
| $X_2$ | $0011$ | $Z_2$ | $0100$ | $X_2Z_2$ | $0111$ |
| $X_3$ | $0110$ | $Z_3$ | $1001$ | $X_3Z_3$ | $1111$ |
| $X_4$ | $1100$ | $Z_4$ | $0010$ | $X_4Z_4$ | $1110$ |
| $X_5$ | $1010$ | $Z_5$ | $0001$ | $X_5Z_5$ | $1011$ |

![The encoding circuit of the five-qubit code.](./5qcec.jpg)

The encoding circuit of the five-qubit code.

We now move on to the implementation of this code.

In [ ]:
def initialize_five_qubits_circuit() -> QuantumCircuit:
  """Initializes a five-qubit circuit, the logical qubit is in position 2.

  Returns:
      QuantumCircuit: The initialized circuit.
  """
  qreg_q = QuantumRegister(9, 'q')
  creg_s = ClassicalRegister(4, 's')
  creg_l = ClassicalRegister(5, 'l')
  circuit = QuantumCircuit(qreg_q, creg_s, creg_l)

  return circuit


def encode_five_qubits(circuit: QuantumCircuit) -> QuantumCircuit:
  """Encodes a circuit with the five-qubit code.

  Args:
      circuit (QuantumCircuit): The circuit to be encoded.

  Returns:
      QuantumCircuit: The encoded circuit
  """
  circuit.h([0, 1, 3, 4])
  
  circuit.cz(0, 1)
  circuit.cx(0, 2)
  circuit.cz(0, 4)
  
  circuit.cz(1, 2)
  circuit.cz(1, 3)
  circuit.cz(1, 4)
  
  circuit.cz(3, 2)
  circuit.cx(4, 2)
  
  return circuit


def extract_syndrome_five_qubits(circuit: QuantumCircuit):
  """Adds the five-qubit syndrome extraction.

  Args:
      circuit (QuantumCircuit): The circuit to add the syndrome extraction in.
  """
  circuit.compose(encode_five_qubits(initialize_five_qubits_circuit()).inverse(), inplace=True)
  circuit.barrier()
  circuit.cx(0, 5)
  circuit.cx(1, 6)
  circuit.cx(3, 7)
  circuit.cx(4, 8)
  
  circuit.barrier()
  

def measure_syndrome_five_qubits(circuit: QuantumCircuit):
  """Adds the five-qubit syndrome measurements.

  Args:
      circuit (QuantumCircuit): The circuit to add the syndrome measurements in.
  """
  circuit.measure(5, 3)
  circuit.measure(6, 2)
  circuit.measure(7, 1)
  circuit.measure(8, 0)
  
  circuit.barrier()

def correct_error_five_qubits(circuit: QuantumCircuit):
  """Adds the five-qubit error correction and decodes accordingly.

  Args:
      circuit (QuantumCircuit): The circuit to add the error correction in.
  """
  # X1 error
  circuit.x(7)
  circuit.x(5)
  circuit.mcx([8, 7, 6, 5], 1)
  circuit.mcx([8, 7, 6, 5], 2)
  circuit.mcx([8, 7, 6, 5], 4)
  circuit.x(7)
  circuit.x(5)
  
  # Z1 error
  circuit.x(8)
  circuit.x(7)
  circuit.x(6)
  circuit.mcx([8, 7, 6, 5], 0)
  circuit.x(8)
  circuit.x(7)
  circuit.x(6)
  
  # X1 Z1 error
  circuit.x(7)
  circuit.mcx([8, 7, 6, 5], 0)
  circuit.mcx([8, 7, 6, 5], 1)
  circuit.mcx([8, 7, 6, 5], 2)
  circuit.mcx([8, 7, 6, 5], 4)
  circuit.x(7)
  
  # X2 error
  circuit.x(6)
  circuit.x(5)
  circuit.mcx([8, 7, 6, 5], 3)
  circuit.mcx([8, 7, 6, 5], 4)
  circuit.h(2)
  circuit.mcx([8, 7, 6, 5], 2)
  circuit.h(2)
  circuit.x(6)
  circuit.x(5)
  
  # Z2 error
  circuit.x(8)
  circuit.x(7)
  circuit.x(5)
  circuit.mcx([8, 7, 6, 5], 1)
  circuit.x(8)
  circuit.x(7)
  circuit.x(5)
  
  # X2 Z2 error
  circuit.x(5)
  circuit.mcx([8, 7, 6, 5], 1)
  circuit.mcx([8, 7, 6, 5], 3)
  circuit.mcx([8, 7, 6, 5], 4)
  circuit.h(2)
  circuit.mcx([8, 7, 6, 5], 2)
  circuit.h(2)
  circuit.x(5)
  
  # X3 error
  circuit.x(8)
  circuit.x(5)
  circuit.mcx([8, 7, 6, 5], 1)
  circuit.mcx([8, 7, 6, 5], 2)
  circuit.mcx([8, 7, 6, 5], 3)
  circuit.x(8)
  circuit.x(5)
  
  # Z3 error
  circuit.x(7)
  circuit.x(6)
  circuit.mcx([8, 7, 6, 5], 0)
  circuit.mcx([8, 7, 6, 5], 4)
  circuit.h(2)
  circuit.mcx([8, 7, 6, 5], 2)
  circuit.h(2)
  circuit.x(7)
  circuit.x(6)
  
  # X3 Z3 error
  circuit.mcx([8, 7, 6, 5], 0)
  circuit.mcx([8, 7, 6, 5], 1)
  circuit.mcx([8, 7, 6, 5], 2)
  circuit.mcx([8, 7, 6, 5], 3)
  circuit.mcx([8, 7, 6, 5], 4)
  circuit.h(2)
  circuit.mcx([8, 7, 6, 5], 2)
  circuit.h(2)
  
  # X4 error
  circuit.x(8)
  circuit.x(7)
  circuit.mcx([8, 7, 6, 5], 0)
  circuit.mcx([8, 7, 6, 5], 1)
  circuit.h(2)
  circuit.mcx([8, 7, 6, 5], 2)
  circuit.h(2)
  circuit.x(8)
  circuit.x(7)
  
  # Z4 error
  circuit.x(8)
  circuit.x(6)
  circuit.x(5)
  circuit.mcx([8, 7, 6, 5], 3)
  circuit.x(8)
  circuit.x(6)
  circuit.x(5)
  
  # X4 Z4 error
  circuit.x(8)
  circuit.mcx([8, 7, 6, 5], 0)
  circuit.mcx([8, 7, 6, 5], 1)
  circuit.mcx([8, 7, 6, 5], 3)
  circuit.h(2)
  circuit.mcx([8, 7, 6, 5], 2)
  circuit.h(2)
  circuit.x(8)
  
  # X5 error
  circuit.x(8)
  circuit.x(6)
  circuit.mcx([8, 7, 6, 5], 0)
  circuit.mcx([8, 7, 6, 5], 2)
  circuit.mcx([8, 7, 6, 5], 3)
  circuit.x(8)
  circuit.x(6)
  
  # Z5 error
  circuit.x(7)
  circuit.x(6)
  circuit.x(5)
  circuit.mcx([8, 7, 6, 5], 4)
  circuit.x(7)
  circuit.x(6)
  circuit.x(5)
  
  # X5 Z5 error
  circuit.x(6)
  circuit.mcx([8, 7, 6, 5], 0)
  circuit.mcx([8, 7, 6, 5], 2)
  circuit.mcx([8, 7, 6, 5], 3)
  circuit.mcx([8, 7, 6, 5], 4)
  circuit.x(6)
  
  circuit.barrier()
  

def measure_qubits_five_qubits(circuit: QuantumCircuit):
  """Measures the logical qubits in a five-qubit code circuit.

  Args:
      circuit (QuantumCircuit): The circuit to add the measurements in
  """
  circuit.measure(0, 8)
  circuit.measure(1, 7)
  circuit.measure(2, 6)
  circuit.measure(3, 5)
  circuit.measure(4, 4)
  circuit.barrier()


circuit = initialize_five_qubits_circuit()
encode_five_qubits(circuit)
inject_error(circuit, 5, ErrorType.BitFlip, qubit_num=0)
inject_error(circuit, 5, ErrorType.PhaseFlip, qubit_num=0)
extract_syndrome_five_qubits(circuit)
measure_syndrome_five_qubits(circuit)
correct_error_five_qubits(circuit)
measure_qubits_five_qubits(circuit)


result = run_and_measure(circuit)

print(result)

Injected X error on qubit 5
Injected Z error on qubit 5
{'00000 1011': 499, '00100 1011': 525}


# Fidelity test
A fidelity test in quantum computing evaluates how accurately a quantum operation or state matches its intended ideal version.
The fidelity between two quantum states $\rho$ and $\sigma$, expressed as density matrices, is commonly defined as:

$$
F(\rho, \sigma) = \Big(\text{Tr}\sqrt{\sqrt{\rho} \sigma}\sqrt{\rho}\Big)^2
$$

We now move on to calculate the fidelity test of the Shor code and the 5-qubit code using a function offered by Qiskit.

In [121]:
# Generate a random state and initialize a pure circuit with it
rand_state = random_statevector(2)
pure_qc = initialize_shor_circuit()
pure_qc.initialize(rand_state, 0)

# Create a Shor circuit and protect the random state with the Shor code
shor_circuit = initialize_shor_circuit()
shor_circuit.initialize(rand_state, 0)
encode_shor(shor_circuit)
inject_error(shor_circuit, 9, ErrorType.BitFlip, qubit_num=4)
inject_error(shor_circuit, 9, ErrorType.PhaseFlip, qubit_num=4)
extract_syndrome_shor_bitflip(shor_circuit)
extract_syndrome_shor_phaseflip(shor_circuit)
correct_error_shor_bitflip(shor_circuit)
correct_error_shor_phaseflip(shor_circuit)
decode_shor(shor_circuit)

# Resetting the syndrome qubits
for i in range(9, 17):
  shor_circuit.reset(i)

# Calculate fidelity with Statevector
pure_sv = Statevector(pure_qc)
shor_sv = Statevector(shor_circuit)

fidelity = state_fidelity(pure_sv, shor_sv)

print("Fidelity is:", fidelity)

Injected X error on qubit 5
Injected Z error on qubit 5
Fidelity is: 1.0


In [120]:
# Initialize a five-qubit circuit with a qubit in a random state
rand_state = random_statevector(2)
pure_qc = initialize_five_qubits_circuit()
pure_qc.initialize(rand_state, 2)

# Create a five-qubit circuit and protect the random state with the five-qubit code
five_qubits_circuit = initialize_five_qubits_circuit()
five_qubits_circuit.initialize(rand_state, 2)
encode_five_qubits(five_qubits_circuit)
inject_error(five_qubits_circuit, 5, ErrorType.BitFlip, qubit_num=0)
inject_error(five_qubits_circuit, 5, ErrorType.PhaseFlip, qubit_num=0)
extract_syndrome_five_qubits(five_qubits_circuit)
correct_error_five_qubits(five_qubits_circuit)

# Resetting the syndrome qubits
for i in range(5, 9):
  five_qubits_circuit.reset(i)

# Calculate fidelity with Statevector
pure_sv = Statevector(pure_qc)
five_qubits_sv = Statevector(five_qubits_circuit)

fidelity = state_fidelity(pure_sv, five_qubits_sv)

print("Fidelity is:", fidelity)

Injected X error on qubit 1
Injected Z error on qubit 1
Fidelity is: 1.0


We can observe that the fidelity nears the value of 1.0 as we use an ideal simulator. This demonstrates that, under ideal conditions, these codes protect a quantum state from an arbitrary single-qubit error.

# Conclusion

The quantum error correction protocols aim to make possible the use of qubits in a reliable way, thus addressing one of the major obstacles in realizing full-scale quantum computers. However, there is a significant trade-off that needs to be considered: the protocols analyzed require a large number of qubits to operate effectively. This inevitably requires a greater computational load, along with the overheads inherent to quantum computing.

Moreover, issues emerge when creating quantum codes due to the no-cloning theorem, the need to address two types of errors, and the wave-function collapse. For the codes analyzed, the quantum information is distributed across an expanded spaces of qubits. Errors are detected through projective stabilizer measurements, whose outcomes are then interpreted to determine the best recovery operation.

In conclusion, many challenges remain to be solved, both at the hardware and software levels, with progress in one area strongly influencing the other.

# References

Core references:

- Roffe, J. *Quantum Error Correction: An Introductory Guide*. Contemporary Physics, 2019.
- Laflamme, R., Miquel, C., Paz, J.P., and Zurek, W.H. *Perfect Quantum Error Correction Code*. Physical Review Letters, 1996.
- Qiskit Textbook (IBM).
- IBM Quantum Learning, *Foundations of Quantum Error Correction*.

For hardware context:

- Wang, X.L., Chen, L.K., Li, W., et al. *Experimental Ten-Photon Entanglement*. Physical Review Letters, 2016.
- Qiang, X., Zhou, X., Wang, J., et al. *Large-Scale Silicon Quantum Photonics Implementing Arbitrary Two-Qubit Processing*. Nature Photonics, 2018.
- Randall, J., Weidt, S., Standing, E.D., et al. *Efficient Preparation and Detection of Microwave Dressed-State Qubits and Qutrits with Trapped Ions*. Physical Review A, 2015.
- Ballance, C., Harty, T., Linke, N., et al. *High-Fidelity Quantum Logic Gates Using Trapped-Ion Hyperfine Qubits*. Physical Review Letters, 2016.
- Brandl, M.F., van Mourik, M.W., Postler, L., et al. *Cryogenic Setup for Trapped Ion Quantum Computing*. Review of Scientific Instruments, 2016.
- Debnath, S., Linke, N.M., Figgatt, C., et al. *Demonstration of a Small Programmable Quantum Computer with Atomic Qubits*. Nature, 2016.

Further readings:

- Wootters, W.K., and Zurek, W.H. *A Single Quantum Cannot Be Cloned*. Nature, 1982.
- Knill, E., and Laflamme, R. *Theory of Quantum Error-Correcting Codes*. Physical Review A, 1997.